In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from pathlib import Path
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.datasets.wrappers.multiview_wrapper import MultiViewWrapper
from cosypose.visualization.multiview import make_cosypose_plots, render_candidates, filter_predictions, get_group_infos
from cosypose.visualization.multiview import make_scene_renderings
import seaborn as sns
from collections import defaultdict
import os
from bokeh.io import show, output_notebook; output_notebook()
from bokeh.plotting import gridplot
import numpy as np
import matplotlib.pyplot as plt
import imageio
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Load results: Choose dataset, result_id

Comment/uncomment (shortcut ctrl+:) blocks of code under each dataset to choose a dataset

In [ ]:
results_dir = LOCAL_DATA_DIR / 'results'

# YCB-Video
# result_id = 'ycbv-n_views=5--8073381555'
# n_views = 5

# scene_ds_name = 'ycbv.test.keyframes'
# det_prefix = 'posecnn_init'
# urdf_ds_name = 'ycbv'

# T-LESS
result_id = 'tless-vivo-n_views=8--2322743008'
n_views = 8
scene_ds_name = 'tless.primesense.test.bop19'
det_prefix = 'pix2pose_detections'
urdf_ds_name = 'tless.cad'
colormap_rgb = None

results = torch.load(results_dir / result_id / 'results.pth.tar')

scene_ds = make_scene_dataset(scene_ds_name)
mv_scene_ds = MultiViewWrapper(scene_ds, n_views)

dict_preds = dict()
for k in ('cand_inputs', 'cand_matched', 'ba_output'):
    dict_preds[k] = results['predictions'][f'{det_prefix}/{k}']

# Choose the group of views for which we want to plot predictions

Print the group ids that correspond to a scene. You can use this to choose a group id if you want to look at a scene in particular.

In [ ]:
# YCB-Video
# scene_id = 55

# TLESS
scene_id = 16

objects = results['predictions'][f'{det_prefix}/scene/objects']
cameras = results['predictions'][f'{det_prefix}/scene/cameras']
print(f"{scene_ds_name} scene {scene_id} has groups: ", np.unique(objects.infos.loc[:, ['scene_id', 'group_id']].set_index('scene_id').loc[scene_id]['group_id']))

In [ ]:
# Example YCB-Video
# group_id = 334

# Example on T-LESS
group_id = 105

scene_id, view_ids = get_group_infos(group_id, mv_scene_ds)
print(f'Making plots for group_id={group_id} (scene_id={scene_id}, view_ids={view_ids}')

# Plot input images / detections / candidates with marked inliers / scene reconstructed / ground truth

In [ ]:
# Note: two distincts objects may have the same color due to the limited size of the color palette
n_views_to_plot = 4
renderer = BulletSceneRenderer(urdf_ds_name)
fig_array = make_cosypose_plots(
    scene_ds,
    scene_id=scene_id, 
    view_ids=view_ids[:n_views_to_plot],
    dict_predictions=dict_preds,
    renderer=renderer,
    use_class_colors_for_3d='ycbv' not in scene_ds_name,
    outlier_color=(1, 1, 1)
)
renderer.disconnect()

show(gridplot(fig_array, sizing_mode='scale_width'))

# Make visualization GIF

In [ ]:
# Note: colors aren't supposed to match the ones from the plots above.
objects = results['predictions'][f'{det_prefix}/scene/objects']
cameras = results['predictions'][f'{det_prefix}/scene/cameras']
objects_, cameras_ = filter_predictions(objects, group_id), filter_predictions(cameras, group_id)

fps = 25
duration = 10
n_images = fps * duration
# n_images = 1  # Uncomment this if you just want to look at one image, generating the gif takes some time
images = make_scene_renderings(objects_, cameras_,
                               urdf_ds_name=urdf_ds_name, 
                               distance=1.3, object_scale=2.0,
                               show_cameras=False, camera_color=(0, 0, 0, 1),
                               theta=np.pi/4, resolution=(640, 480),
                               object_id_ref=0, 
                               colormap_rgb=defaultdict(lambda: [1, 1, 1, 1]) if 'ycb' in scene_ds_name else None,
                               angles=np.linspace(0, 2*np.pi, n_images))
save_path = f'gifs/scene_ds={scene_ds_name}-scene={scene_id}-nviews={n_views}-scene_group={group_id}.gif'
print(f"Save location: {Path(save_path).resolve()}")
imageio.mimsave(save_path, images, fps=fps)
plt.imshow(images[0])